In [1]:
from env.cartpolecont import *

import gym
import time
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt

from misc.random_agent import *
from control.lqr_agent import *

# environment to be recorded
env = gym.make('CartPoleCont-v0')
sim_env = gym.make('CartPoleCont-v0')
sim_env.reset()

array([-0.13015236, -0.02796216, -0.16196381,  0.10385134])

In [2]:
# envtr: env to be recorded
def simulate(envtr, agent_type, T=100):
    envtr._max_episode_steps = 500
    env = gym.wrappers.Monitor(envtr, "videos", force=True)
    env.reset()
    agent = agent_type(env)
    for i in range(T):
        env.render(mode='human')
        time.sleep(0.02)
        a = np.clip(agent.get_action(), -1, 1)
        _, _, done, _ = env.step(a) # take a random action
        if done:
            break
    
    print("Episode ended after {} timesteps".format(i+1))
    env.stats_recorder.save_complete()
    env.stats_recorder.done = True
    env.close()
    envtr.reset()
    envtr.close()

## Random Agent

In [3]:
simulate(env, RandomAgent)

Episode ended after 8 timesteps


## LQR Agent

In [4]:
simulate(env, lambda env:LQRAgent(env, sim_env))

Episode ended after 100 timesteps


In [5]:
# from matplotlib import animation

# def save_frames_as_gif(frames, path='./', filename='gym_animation.gif'):

#     #Mess with this to change frame size
#     plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)

#     patch = plt.imshow(frames[0])
#     plt.axis('off')

#     def animate(i):
#         patch.set_data(frames[i])

#     anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
#     anim.save(path + filename, writer='imagemagick', fps=40)

# env.reset()
# sim_env.reset()
# frames = []
# agent = LQRAgent(env, sim_env)
# for t in range(100):
#     #Render to frames buffer
#     frames.append(env.render(mode="rgb_array"))
#     action = np.clip(agent.get_action(), -1, 1)
#     _, _, done, _ = env.step(action)
#     if done:
#         break
# env.close()
# save_frames_as_gif(frames, filename='lqr_cartpole.gif')